<a href="https://colab.research.google.com/github/2303A52083/23CSBTB39-AIML/blob/main/AIML_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Implement the AI Game Strategy

Part 1 –(a). Install the Python Libraries required for Game Strategy.

Part 2 – Implement the Game Strategy Algorithms

Part 3 – Implement the Game Strategy using TicTocToe.

Part 4 – Evaluate the AI Game Strategy using TicTocToe.


In [8]:
from collections import namedtuple
import random
import math
import functools
cache = functools.lru_cache(maxsize=10**6)
class Game:
    def actions(self, state):
        raise NotImplementedError
    def result(self, state, move):
        raise NotImplementedError
    def is_terminal(self, state):
        return not self.actions(state)
    def utility(self, state, player):
        raise NotImplementedError
    def play_game(self, game, strategies: dict, verbose=False):
        state = game.initial
        while not game.is_terminal(state):
            player = state.to_move
            move = strategies[player](game, state)
            state = game.result(state, move)
            if verbose:
                print('Player', player, 'move:', move)
                print(state)
        return state
def minimax_search(game, state):
    player = state.to_move
    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move
    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move
    return max_value(state)
def alphabeta_search(game, state):
    player = state.to_move
    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
            alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move
    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
            beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move
    return max_value(state, -math.inf, math.inf)
class TicTacToe(Game):
    def __init__(self, height=3, width=3, k=3):
        self.k = k
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)
    def actions(self, board):
        return self.squares - set(board.board)
    def result(self, board, square):
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = self.k_in_row(board, player, square, self.k)
        board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board
    def utility(self, board, player):
        return board.utility if player == 'X' else -board.utility
    def is_terminal(self, board):
        return board.utility != 0 or len(self.actions(board)) == 0
    def display(self, board):
        print(board)
    def k_in_row(self, board, player, square, k):
        def in_row(x, y, dx, dy):
            return 0 if board.board.get((x, y), None) != player else 1 + in_row(x + dx, y + dy, dx, dy)
        return any(
            in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
            for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1))
        )
class Board:
    def __init__(self, width=8, height=8, to_move=None, utility=0, **kwargs):
        self.width = width
        self.height = height
        self.to_move = to_move
        self.utility = utility
        self.board = kwargs
    def new(self, changes: dict, **kwargs) -> 'Board':
        board = Board(width=self.width, height=self.height, **kwargs)
        board.board.update(self.board)
        board.board.update(changes)
        return board
    def missing(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off
    def __hash__(self):
        return hash(tuple(sorted(self.board.items()))) + hash(self.to_move)
    def __repr__(self):
        def row(y):
            return ' '.join(self.board.get((x, y), ' ') for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) + '\n'
def random_player(game, state):
    return random.choice(list(game.actions(state)))
def player(search_algorithm):
    return lambda game, state: search_algorithm(game, state)[1]
def evaluate_strategies():
    game = TicTacToe()
    strategies = dict(X=random_player, O=player(alphabeta_search))
    final_state = game.play_game(game, strategies, verbose=True)
    print("Final utility for Strategy 1 (X=random, O=alphabeta):", final_state.utility)
    strategies = dict(X=player(alphabeta_search), O=player(minimax_search))
    final_state = game.play_game(game, strategies, verbose=True)
    print("Final utility for Strategy 2 (X=alphabeta, O=minimax):", final_state.utility)
evaluate_strategies()


Player X move: (2, 0)
    X
     
     

Player O move: (1, 1)
    X
  O  
     

Player X move: (0, 1)
    X
X O  
     

Player O move: (1, 2)
    X
X O  
  O  

Player X move: (0, 2)
    X
X O  
X O  

Player O move: (1, 0)
  O X
X O  
X O  

Final utility for Strategy 1 (X=random, O=alphabeta): -1
Player X move: (0, 1)
     
X    
     

Player O move: (2, 1)
     
X   O
     

Player X move: (1, 2)
     
X   O
  X  

Player O move: (0, 0)
O    
X   O
  X  

Player X move: (1, 1)
O    
X X O
  X  

Player O move: (1, 0)
O O  
X X O
  X  

Player X move: (2, 0)
O O X
X X O
  X  

Player O move: (0, 2)
O O X
X X O
O X  

Player X move: (2, 2)
O O X
X X O
O X X

Final utility for Strategy 2 (X=alphabeta, O=minimax): 0
